- DOING
    - 使用`./trial`目录下的txt文件模拟流程（可以按GNSS时间戳判断是否缺数据）
    - 使用提供的**trialname**测试
        - `T62288821d9d01`
        - `T62288821d9d02`

In [1]:
import requests
import numpy as np
import torch
import time
import copy
from parse import parse

from spd_dnn import *
from data_struct import *
from ftr_generator import *
from coord_utils import *


# 算法流程

## 航向推算

In [2]:
def infer_bearing(sec_gyro_unit_list, crnt_brng, collect_hz=250):
    gy_ys = [u.sen_y for u in sec_gyro_unit_list]
    bearing_diff = np.sum(gy_ys) / collect_hz * -1.0
    return (crnt_brng + bearing_diff) % 360.0

## 速度推算

In [3]:
ftr_hz = 25
collect_hz = 250

model = load_spd_dnn(trained_model='./spd_dnn_weight_ftrHz%d.pt' % ftr_hz,
                     ftr_hz=ftr_hz, collect_hz=collect_hz)
model.eval()

ftr_gen = ftrGenerator(ftr_hz=ftr_hz, collect_hz=collect_hz)

Load trained model: ./spd_dnn_weight_ftrHz25.pt


In [4]:
def calibrate_sample_freq(imu_unit_list, collect_hz=250):
    if len(imu_unit_list) < collect_hz:
        imu_unit_list.extend([copy.deepcopy(imu_unit_list[-1])] * (collect_hz - len(imu_unit_list)))
    elif len(imu_unit_list) > collect_hz:
        imu_unit_list = imu_unit_list[-collect_hz:]
    else:
        pass
    return imu_unit_list


def infer_speed(sec_acc_unit_list, sec_gyro_unit_list, crnt_spd,
                model=None, ftr_gen=None, collect_hz=250):
    sec_acc_unit_list = calibrate_sample_freq(sec_acc_unit_list, collect_hz=collect_hz)
    sec_gyro_unit_lis = calibrate_sample_freq(sec_gyro_unit_list, collect_hz=collect_hz)
    acc_ftr_list = ftr_gen.calc_imu_sec_ftr(sec_acc_unit_list)
    gy_ftr_list = ftr_gen.calc_imu_sec_ftr(sec_gyro_unit_lis)

    acc = torch.tensor([[acc_ftr_list]], dtype=torch.float).permute(0, 2, 1)
    gy = torch.tensor([[gy_ftr_list]], dtype=torch.float).permute(0, 2, 1)
    init_spd = torch.tensor([[[crnt_spd]]], dtype=torch.float)
    pred = model(acc, gy, init_spd=init_spd)
    spd_diff = pred.cpu().detach().squeeze().numpy().tolist()

    return max(0.0, min(33.0, crnt_spd + spd_diff))

# 服务端配置及请求

- 使用官方服务器测试

In [36]:
server = 'https://evaal.aaloa.org/evaalapi/'
trialname = 'T62288821d9d02'

estfmt = "{pts:.3f},{c:.3f},{h:.3f},{s:.3f},{pos}"
statefmt = "{trialts:.3f},{rem:.3f},{V:.3f},{S:.3f},{p:.3f},{h:.3f},{pts:.3f},{pos}"

- 使用本地服务器测试

In [37]:
# server = 'http://127.0.0.1:5000/evaalapi/'
# trialname = 'demo1'

# estfmt = "{pts:.3f},{c:.3f},{h:.3f},{s:.3f},{pos}"
# statefmt = "{trialts:.3f},{rem:.3f},{V:.3f},{S:.3f},{p:.3f},{h:.3f},{pts:.3f},{pos}"

In [25]:
def do_state():
    r = requests.get(server + trialname + '/state')
    print(r.text)
    s = parse(statefmt, r.text)
    return s.named
    
    
def do_reload():
    r = requests.get(server + trialname + '/reload')
    s = parse(statefmt, r.text)
    print(s.named)

In [26]:
def parse_response(text):
    sen_str_list = text.splitlines()
    acc_unit_list, gy_unit_list = [], []
    gnss_unit = None
    for sen_str in sen_str_list:
        if sen_str.startswith('ACCE'):
            acc_unit_list.append(acceUnit(sen_str))
        elif sen_str.startswith('GYRO'):
            gy_unit_list.append(gyroUnit(sen_str))
        elif sen_str.startswith('GNSS'):
            gnss_unit = gnssUnit(sen_str)
        else:
            continue
    return acc_unit_list, gy_unit_list, gnss_unit


def estimate_next_loc(acc_unit_list, gy_unit_list, gnss_unit, 
                      model=None, ftr_gen=None, ftr_hz=50, collect_hz=250):
    crnt_brng = gnss_unit.bearing
    crnt_spd = gnss_unit.loc_speed
    if crnt_spd < 0:
        crnt_spd = gnss_unit.speed
    
    next_brng = infer_bearing(gy_unit_list, crnt_brng, collect_hz=collect_hz)
    next_spd = infer_speed(acc_unit_list, gy_unit_list, crnt_spd, 
                           model=model, ftr_gen=ftr_gen, collect_hz=collect_hz)
    
    next_lng, next_lat = geo_util.position_from_angle_and_dist(gnss_unit.lng, gnss_unit.lat, 
                                                               next_brng, next_spd)
    return next_lng, next_lat, next_spd, next_brng


def estimate_next_loc_by_pred(acc_unit_list, gy_unit_list, pred_loc, 
                              model=None, ftr_gen=None, ftr_hz=50, collect_hz=250):
    crnt_lng, crnt_lat, crnt_spd, crnt_brng = pred_loc
    next_brng = infer_bearing(gy_unit_list, crnt_brng, collect_hz=collect_hz)
    next_spd = infer_speed(acc_unit_list, gy_unit_list, crnt_spd, 
                           model=model, ftr_gen=ftr_gen, collect_hz=collect_hz)
    next_lng, next_lat = geo_util.position_from_angle_and_dist(crnt_lng, crnt_lat, 
                                                               next_brng, next_spd)
    return next_lng, next_lat, next_spd, next_brng
    
    
def estimate_next_loc_by_dft(crnt_lng, crnt_lat, crnt_spd, crnt_brng):
    next_brng, next_spd = crnt_brng, crnt_spd
    next_lng, next_lat = geo_util.position_from_angle_and_dist(crnt_lng, crnt_lat, 
                                                               next_brng, next_spd)
    return next_lng, next_lat, next_spd, next_brng
    

In [38]:
init_pos = None  # 车辆初始位置
altitude = 0.0  # 高程
pre_gnss = None  # 上一时刻GNSS
pred_loc = None
next_position = None
gt_gnss_unit_list = []

sec_cnt = 0

do_reload()
while True:
    s = do_state()
    if s['trialts'] == 0.0:
        print('testing start...')
        try:
            lat, lng, altitude = [float(x) for x in s['pos'].split(',')]
        except ValueError as e:
            lat, lng, altitude = [float(x) for x in s['pos'].split(';')]
        init_pos = [lng, lat]
    elif s['trialts'] < 0:
        print('testing timeout...')
        break
    else:
        pass
    
    if next_position is None:
        r = requests.get(server + trialname + '/nextdata?horizon=1.0')
    else:
        r = requests.get(server + trialname + \
                         '/nextdata?horizon=1.0&position=%s;%.1f' % (next_position, altitude))
    
    acc_unit_list, gy_unit_list, gnss_unit = parse_response(r.text)
    
    if gnss_unit is None:
        print(len(acc_unit_list))
        print(len(gy_unit_list))
        
    if gnss_unit is not None:
        gt_gnss_unit_list.append(gnss_unit)
        
    if len(acc_unit_list) < 10 and gnss_unit is None:
        if pre_gnss is not None:
            lng, lat, spd, brng = pre_gnss.lng, pre_gnss.lat, pre_gnss.loc_speed, pre_gnss.bearing
            if spd < 0:
                spd = pre_gnss.speed
            next_lng, next_lat, \
            next_spd, next_brng = estimate_next_loc_by_dft(lng, lat, spd, brng)
            next_position = '%.6f;%.6f' % (next_lat, next_lng)
            pred_loc = [next_lng, next_lat, next_spd, next_brng]
        elif pred_loc is not None:
            next_lng, next_lat, \
            next_spd, next_brng = estimate_next_loc_by_dft(pred_loc[0], pred_loc[1], 
                                                           pred_loc[2], pred_loc[3])
            next_position = '%.6f;%.6f' % (next_lat, next_lng)
            pred_loc = [next_lng, next_lat, next_spd, next_brng]
        else:
            pass
        time.sleep(0.01)
        sec_cnt += 1
        continue
    
    if gnss_unit is not None:
        gnss_unit.pre_gnss = pre_gnss
        pre_gnss = gnss_unit
        next_position = '%.6f;%.6f' % (gnss_unit.lat, gnss_unit.lng)
    elif pre_gnss is not None:
        next_lng, next_lat, \
        next_spd, next_brng = estimate_next_loc(acc_unit_list, gy_unit_list, pre_gnss, 
                                                model=model, ftr_gen=ftr_gen, 
                                                ftr_hz=ftr_hz, collect_hz=collect_hz)
        next_position = '%.6f;%.6f' % (next_lat, next_lng)
        pred_loc = [next_lng, next_lat, next_spd, next_brng]
        pre_gnss = None
    elif pred_loc is not None:
        next_lng, next_lat, \
        next_spd, next_brng = estimate_next_loc_by_pred(acc_unit_list, gy_unit_list, pred_loc, 
                                                        model=model, ftr_gen=ftr_gen, 
                                                        ftr_hz=ftr_hz, collect_hz=collect_hz)
        next_position = '%.6f;%.6f' % (next_lat, next_lng)
        pred_loc = [next_lng, next_lat, next_spd, next_brng]
    else:
        pass
        
    time.sleep(0.01)
    sec_cnt += 1
#     if sec_cnt == 2500:
#         break

{'trialts': 0.0, 'rem': -1.0, 'V': 9.0, 'S': 45.0, 'p': 0.0, 'h': 0.0, 'pts': 0.0, 'pos': '0,0,0'}
0.000,-1.000,9.000,45.000,0.000,0.000,0.000,0,0,0
testing start...
463414.000,53.205,9.000,45.000,1661970438.318,1.000,463413.000,0,0,0
463415.000,53.022,9.000,45.000,1661970439.894,1.000,463414.000,40.225612;116.183918;0.0
463416.000,52.526,9.000,45.000,1661970441.922,1.000,463415.000,40.225617;116.183916;0.0
463417.000,52.753,9.000,45.000,1661970444.704,1.000,463416.000,40.225599;116.183861;0.0
463418.000,52.578,9.000,45.000,1661970446.734,1.000,463417.000,40.225643;116.183909;0.0
463419.000,52.702,9.000,45.000,1661970449.293,1.000,463418.000,40.225656;116.183889;0.0
463420.000,52.710,9.000,45.000,1661970451.555,1.000,463419.000,40.225645;116.183910;0.0
463421.000,52.386,9.000,45.000,1661970454.157,1.000,463420.000,40.225642;116.183891;0.0
463422.000,52.652,9.000,45.000,1661970456.627,1.000,463421.000,40.225642;116.183879;0.0
250
250
463423.000,51.984,9.000,45.000,1661970459.300,1.000,4

463506.000,52.881,9.000,45.000,1661970661.728,1.000,463505.000,40.225570;116.183876;0.0
463507.000,52.696,9.000,45.000,1661970663.768,1.000,463506.000,40.225570;116.183876;0.0
463508.000,52.978,9.000,45.000,1661970665.851,1.000,463507.000,40.225570;116.183876;0.0
463509.000,52.589,9.000,45.000,1661970667.928,1.000,463508.000,40.225570;116.183876;0.0
463510.000,52.525,9.000,45.000,1661970670.650,1.000,463509.000,40.225570;116.183876;0.0
463511.000,51.658,9.000,45.000,1661970672.972,1.000,463510.000,40.225570;116.183876;0.0
463512.000,52.927,9.000,45.000,1661970676.147,1.000,463511.000,40.225570;116.183876;0.0
463513.000,51.873,9.000,45.000,1661970679.007,1.000,463512.000,40.225570;116.183876;0.0
463514.000,52.969,9.000,45.000,1661970682.015,1.000,463513.000,40.225570;116.183876;0.0
248
249
463515.000,51.867,9.000,45.000,1661970684.250,1.000,463514.000,40.225570;116.183876;0.0
463516.000,52.936,9.000,45.000,1661970687.333,1.000,463515.000,40.225570;116.183876;0.0
463517.000,52.889,9.000,

KeyboardInterrupt: 

In [39]:
# est_fd = open('./ScoringResult01.csv', 'w')

# r = requests.get(server + trialname + '/estimates')
# for i, line in enumerate(r.text.splitlines()):
#     if i == 0:
#         continue
#     s = parse(estfmt, line)
#     ts = int(s.named['pts'])
#     lat, lng, altitude = [float(x) for x in s.named['pos'].split(';')]
#     est_fd.write('%d,%.6f,%.6f,%.1f\n' % (ts, lat, lng, altitude))

In [40]:
r = requests.get(server + trialname + '/estimates')
print(len(r.text.splitlines()))

114


In [41]:
# print(r.text)

## 评估

In [42]:
ts_estimate_loc_map = {}

r = requests.get(server + trialname + '/estimates')
for i, line in enumerate(r.text.splitlines()):
    if i in {0, 1}:
        continue
    _items = line.strip().split(',')
    ts = int(float(_items[0]))
    loc_str = _items[-1]
    lat, lng = float(loc_str.split(';')[0]), float(loc_str.split(';')[1])
    lng, lat = geo_util.wgs84_to_gcj02(lng, lat)
    ts_estimate_loc_map[ts] = [lng, lat]
    
ts_list = sorted(ts_estimate_loc_map.keys(), reverse=False)
with open('offline_estimate.csv', 'w') as fd:
    fd.write('ts,lng,lat\n')
    for i, ts in enumerate(ts_list):
        lng, lat = ts_estimate_loc_map[ts]
        if i > 0:
            ts = ts - ts_list[0]
        fd.write('%d,%.6f,%.6f\n' % (ts, lng, lat))

In [43]:
ts_gnss_loc_map = {}

for i, gnss in enumerate(gt_gnss_unit_list):
    ts = int(round(gnss.app_ts, 0))
    lng, lat = gnss.lng_gcj02, gnss.lat_gcj02
    ts_gnss_loc_map[ts] = [lng, lat]
    
ts_list = sorted(ts_gnss_loc_map.keys(), reverse=False)
with open('offline_gnss.csv', 'w') as fd:
    fd.write('ts,lng,lat\n')
    for i, ts in enumerate(ts_list):
        lng, lat = ts_gnss_loc_map[ts]
        if i > 0:
            ts = ts - ts_list[0]
        fd.write('%d,%.6f,%.6f\n' % (ts, lng, lat))

In [44]:
len(ts_gnss_loc_map)

109

In [45]:
dist_err_list = []

for ts, (lng, lat) in ts_estimate_loc_map.items():
    if ts not in ts_gnss_loc_map and ts + 1 in ts_gnss_loc_map:
        dist_err = geo_util.distance(lng, lat, 
                                     ts_gnss_loc_map[ts + 1][0], ts_gnss_loc_map[ts + 1][1])
        dist_err_list.append(dist_err)

In [47]:
print(np.percentile(dist_err_list, 25))
print(np.percentile(dist_err_list, 50))
print(np.percentile(dist_err_list, 75))

0.679057
1.358114
2.6646455


In [46]:
dist_err_list

[3.971177, 1.358114, 0.0]